In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [5]:
from sklearn.model_selection import train_test_split
# Выделение целевой переменной
target = data['MEDV']

# Разделение данных на обуч. и тест. выборки
X_train, X_test, y_train, y_test = train_test_split(data.drop(['MEDV'], axis=1), target, test_size=0.2, random_state=42)

print(f"Обучающая выборка: {X_train.shape}, тестовая выборка: {X_test.shape}")

Обучающая выборка: (404, 13), тестовая выборка: (102, 13)


In [6]:
from sklearn.model_selection import train_test_split
# Выделение целевой переменной
y = data['MEDV']

# Остальные колонки считаем признаковыми
X = data.drop(['MEDV'], axis=1)

# Разделение данных на обуч. и тест. выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training dataset:", X_train.shape, y_train.shape)
print("Testing dataset:", X_test.shape, y_test.shape)

Training dataset: (404, 13) (404,)
Testing dataset: (102, 13) (102,)


2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

# Обучение моделей
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()

# Обучение моделей на обучающей выборке
lr.fit(X_train, y_train)
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

# Вычисление R2 на тестовой выборке
lr_r2 = r2_score(y_test, lr.predict(X_test))
ridge_r2 = r2_score(y_test, ridge.predict(X_test))
lasso_r2 = r2_score(y_test, lasso.predict(X_test))

print(f"R2 линейной регрессии: {lr_r2:.4f}")
print(f"R2 регрессии с Ridge: {ridge_r2:.4f}")
print(f"R2 регрессии с Lasso: {lasso_r2:.4f}")

R2 линейной регрессии: 0.6685
R2 регрессии с Ridge: 0.6660
R2 регрессии с Lasso: 0.6669


In [8]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Обучение линейной регр.
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Обучение Ridge регр.
rid_reg = Ridge()
rid_reg.fit(X_train, y_train)

# Обучение Lasso регр.
las_reg = Lasso()
las_reg.fit(X_train, y_train)

# Вычисление R2 для каждой модели на тестовой выборке
linear_r2 = r2_score(y_test, lin_reg.predict(X_test))
ridge_r2 = r2_score(y_test, rid_reg.predict(X_test))
lasso_r2 = r2_score(y_test, las_reg.predict(X_test))

print("Линейная регрессия:", linear_r2)
print("Ridge регрессия:", ridge_r2)
print("Lasso регрессия:", lasso_r2)

Линейная регрессия: 0.6684825753971593
Ridge регрессия: 0.6659608075261694
Lasso регрессия: 0.6668687223368213


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [10]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

# Подбор параметров для Ridge регр. с помощью GridSearchCV
param_grid = {'alpha': np.logspace(-5, 5, 11)}
ridge_cv = GridSearchCV(Ridge(), param_grid, cv=5)
ridge_cv.fit(X_train, y_train)
ridge_best_model = ridge_cv.best_estimator_
ridge_r2 = r2_score(y_test, ridge_best_model.predict(X_test))
print("Ridge регрессия (GridSearchCV):", ridge_r2)

# Подбор параметров для Ridge регр. с помощью RidgeCV
ridge_cv_mod = RidgeCV(alphas=np.logspace(-5, 5, 11))
ridge_cv_mod.fit(X_train, y_train)
ridge_r2_cv = r2_score(y_test, ridge_cv_mod.predict(X_test))
print("Ridge регрессия (RidgeCV):", ridge_r2_cv)

# Подбор параметров для Lasso регр. с помощью GridSearchCV
param_grid = {'alpha': np.logspace(-5, 5, 11)}
lasso_cv = GridSearchCV(Lasso(), param_grid, cv=5)
lasso_cv.fit(X_train, y_train)
lasso_best_model = lasso_cv.best_estimator_
lasso_r2 = r2_score(y_test, lasso_best_model.predict(X_test))
print("Lasso регрессия (GridSearchCV):", lasso_r2)

# Подбор параметров для Lasso регр. с помощью LassoCV
lasso_cv_mod = LassoCV(alphas=np.logspace(-5, 5, 11))
lasso_cv_mod.fit(X_train, y_train)
lasso_r2_cv = r2_score(y_test, lasso_cv_mod.predict(X_test))
print("Lasso регрессия (LassoCV):", lasso_r2_cv)

Ridge регрессия (GridSearchCV): 0.6684825680074257
Ridge регрессия (RidgeCV): 0.6684745342323266
Lasso регрессия (GridSearchCV): 0.6684829595885678
Lasso регрессия (LassoCV): 0.6684829595885678


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Подготовка стандартных моделей
standard_pipe = make_pipeline(StandardScaler(), Ridge())
minmax_pipe = make_pipeline(MinMaxScaler(), Ridge())

# Обучение моделей
standard_pipe.fit(X_train, y_train)
minmax_pipe.fit(X_train, y_train)

# Вычисление R2 на тестовой выборке
standard_r2 = r2_score(y_test, standard_pipe.predict(X_test))
minmax_r2 = r2_score(y_test, minmax_pipe.predict(X_test))

print(f"R2 стандартной регрессии (с масштабированием): {standard_r2:.4f}")
print(f"R2 стандартной регрессии (с масштабированием MinMaxScaler): {minmax_r2:.4f}")

R2 стандартной регрессии (с масштабированием): 0.6682
R2 стандартной регрессии (с масштабированием MinMaxScaler): 0.6762


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

# Обучение моделей с различными методами масштабирования
lin_reg = LinearRegression()
ridge_reg = Ridge()
lasso_reg = Lasso()

# Обучение StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

# Создание pipelines
lin_reg_scaled = make_pipeline(scaler, lin_reg)
ridge_reg_scaled = make_pipeline(scaler, ridge_reg)
lasso_reg_scaled = make_pipeline(scaler, lasso_reg)

# Обучение моделей
lin_reg_scaled.fit(X_train, y_train)
ridge_reg_scaled.fit(X_train, y_train)
lasso_reg_scaled.fit(X_train, y_train)

# Вычисление R2 для каждой модели на тестовой выборке
linear_r2 = r2_score(y_test, lin_reg.predict(X_test))
linear_r2_scaled = r2_score(y_test, lin_reg_scaled.predict(X_test))

ridge_r2 = r2_score(y_test, ridge_reg.predict(X_test))
ridge_r2_scaled = r2_score(y_test, ridge_reg_scaled.predict(X_test))

lasso_r2 = r2_score(y_test, lasso_reg.predict(X_test))
lasso_r2_scaled = r2_score(y_test, lasso_reg_scaled.predict(X_test))

print("Линейная регрессия:", linear_r2, "| Линейная регрессия (масштаб):", linear_r2_scaled)
print("Ridge регрессия:", ridge_r2, "| Ridge регрессия (масштаб):", ridge_r2_scaled)
print("Lasso регрессия:", lasso_r2, "| Lasso регрессия (масштаб):", lasso_r2_scaled)

Линейная регрессия: -4148.942930338001 | Линейная регрессия (масштаб): 0.6684825753971608
Ridge регрессия: -3726.3240267139986 | Ridge регрессия (масштаб): 0.668190107677443
Lasso регрессия: -215.0515595121212 | Lasso регрессия (масштаб): 0.6240447523478461


C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

# Обучение моделей с различными методами масштабирования
lin_reg = LinearRegression()
ridge_reg = Ridge()
lasso_reg = Lasso()

# Обучение StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

# Создание pipelines
lin_reg_scaled = make_pipeline(scaler, lin_reg)
ridge_reg_scaled = make_pipeline(scaler, ridge_reg)
lasso_reg_scaled = make_pipeline(scaler, lasso_reg)

# Обучение моделей
lin_reg_scaled.fit(X_train, y_train)
ridge_reg_scaled.fit(X_train, y_train)
lasso_reg_scaled.fit(X_train, y_train)

# Вычисление R2 для каждой модели на тестовой выборке
linear_r2 = r2_score(y_test, lin_reg.predict(X_test))
linear_r2_scaled = r2_score(y_test, lin_reg_scaled.predict(X_test))

ridge_r2 = r2_score(y_test, ridge_reg.predict(X_test))
ridge_r2_scaled = r2_score(y_test, ridge_reg_scaled.predict(X_test))

lasso_r2 = r2_score(y_test, lasso_reg.predict(X_test))
lasso_r2_scaled = r2_score(y_test, lasso_reg_scaled.predict(X_test))

print("Линейная регрессия:", linear_r2, "| Линейная регрессия (масштаб):", linear_r2_scaled)
print("Ridge регрессия:", ridge_r2, "| Ridge регрессия (масштаб):", ridge_r2_scaled)
print("Lasso регрессия:", lasso_r2, "| Lasso регрессия (масштаб):", lasso_r2_scaled)

Линейная регрессия: -4148.942930338001 | Линейная регрессия (масштаб): 0.6684825753971608
Ridge регрессия: -3726.3240267139986 | Ridge регрессия (масштаб): 0.668190107677443
Lasso регрессия: -215.0515595121212 | Lasso регрессия (масштаб): 0.6240447523478461


C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
C:\Anaconda3\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [14]:

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Подготовка стандартных моделей
standard_pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2), Ridge())
minmax_pipe = make_pipeline(MinMaxScaler(), PolynomialFeatures(degree=2), Ridge())

# Обучение моделей
standard_pipe.fit(X_train, y_train)
minmax_pipe.fit(X_train, y_train)

# Вычисление R2 на тестовой выборке
standard_r2 = r2_score(y_test, standard_pipe.predict(X_test))
minmax_r2 = r2_score(y_test, minmax_pipe.predict(X_test))


print(f"R2 стандартной регрессии (с масштабированием и полиномиальными признаками): {standard_r2:.4f}")
print(f"R2 стандартной регрессии (с масштабированием MinMaxScaler и полиномиальными признаками): {minmax_r2:.4f}")

R2 стандартной регрессии (с масштабированием и полиномиальными признаками): 0.8171
R2 стандартной регрессии (с масштабированием MinMaxScaler и полиномиальными признаками): 0.8299


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Создание полиномиальных фичей
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X_scaled)

# Модель Ridge
ridge = Ridge(alpha=1.0)
ridge_pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree=4), Ridge(alpha=1.0))
ridge_preds = ridge_pipe.fit(X, y).predict(X)
ridge_r2 = r2_score(y, ridge_preds)
print("Ridge (без Poly): R^2 =", ridge_r2)

# Модель Lasso
lasso = Lasso(alpha=0.1)
lasso_pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree=4), Lasso(alpha=0.1))
lasso_preds = lasso_pipe.fit(X, y).predict(X)
lasso_r2 = r2_score(y, lasso_preds)
print("Lasso (без Poly): R^2 =", lasso_r2)

# Модели с использованием PolynomialFeatures
ridge_poly = Ridge(alpha=1.0)
ridge_poly_pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree=4), Ridge(alpha=1.0))
ridge_poly_preds = ridge_poly_pipe.fit(X_scaled, y).predict(X_scaled)
ridge_poly_r2 = r2_score(y, ridge_poly_preds)
print("Ridge (с Poly): R^2 =", ridge_poly_r2)

lasso_poly = Lasso(alpha=0.1)
lasso_poly_pipe = make_pipeline(StandardScaler(), PolynomialFeatures(degree=4), Lasso(alpha=0.1))
lasso_poly_preds = lasso_poly_pipe.fit(X_scaled, y).predict(X_scaled)
lasso_poly_r2 = r2_score(y, lasso_poly_preds)
print("Lasso (с Poly): R^2 =", lasso_poly_r2)

Ridge (без Poly): R^2 = 0.995880282578042


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.052e+03, tolerance: 4.272e+00
  model = cd_fast.enet_coordinate_descent(


Lasso (без Poly): R^2 = 0.9359910763646421
Ridge (с Poly): R^2 = 0.9958802825780452
Lasso (с Poly): R^2 = 0.9359910763646421


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.052e+03, tolerance: 4.272e+00
  model = cd_fast.enet_coordinate_descent(


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:
# Создание pipeline с разными методами масштабирования и типами регуляризации
scaler_pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=2, include_bias=False))])
minmax_pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures(degree=2, include_bias=False))])

ridge_reg_scaled = make_pipeline(scaler_pipe, Ridge())
ridge_reg_scaled_minmax = make_pipeline(minmax_pipe, Ridge())

lasso_reg_scaled = make_pipeline(scaler_pipe, Lasso())
lasso_reg_scaled_minmax = make_pipeline(minmax_pipe, Lasso())

# Параметры для поиска
param_grid = [{'estimator__alpha': np.logspace(-5, 5, 11)},
              {'estimator__alpha': np.logspace(-5, 5, 11), 'estimator__polynomialfeatures__degree': range(1, 4)}]

# Настройка GridSearchCV для каждого Pipeline
ridge_gridsearch = GridSearchCV(ridge_reg_scaled, param_grid, scoring='r2', cv=5)
ridge_gridsearch_minmax = GridSearchCV(ridge_reg_scaled_minmax, param_grid, scoring='r2', cv=5)

lasso_gridsearch = GridSearchCV(lasso_reg_scaled, param_grid, scoring='r2', cv=5)
lasso_gridsearch_minmax = GridSearchCV(lasso_reg_scaled_minmax, param_grid, scoring='r2', cv=5)

# Запуск GridSearchCV для каждого Pipeline
ridge_gridsearch.fit(X_train, y_train)
ridge_gridsearch_minmax.fit(X_train, y_train)

lasso_gridsearch.fit(X_train, y_train)
lasso_gridsearch_minmax.fit(X_train, y_train)

# Получение лучших моделей и их R2
ridge_best_model = ridge_gridsearch.best_estimator_
ridge_best_model_minmax = ridge_gridsearch_minmax.best_estimator_
ridge_r2 = r2_score(y_test, ridge_best_model.predict(X_test))
ridge_r2_minmax = r2_score(y_test, ridge_best_model_minmax.predict(X_test))

lasso_best_model = lasso_gridsearch.best_estimator_
lasso_best_model_minmax = lasso_gridsearch_minmax.best_estimator_
lasso_r2 = r2_score(y_test, lasso_best_model.predict(X_test))
lasso_r2_minmax = r2_score(y_test, lasso_best_model_minmax.predict(X_test))

# Получение и вывод результатов
print("Лучшая модель Ridge (масштаб):")
print("\t", ridge_best_model.get_params())
print("\t R2:", ridge_r2)

print("Лучшая модель Ridge (MinMaxScale):")
print("\t", ridge_best_model_minmax.get_params())
print("\t R2:", ridge_r2_minmax)

print("Лучшая модель Lasso (масштаб):")
print("\t", lasso_best_model.get_params())
print("\t R2:", lasso_r2)

print("Лучшая модель Lasso (MinMaxScale):")
print("\t", lasso_best_model_minmax.get_params())
print("\t R2:", lasso_r2_minmax)

http://archive.ics.uci.edu/ml/datasets/Adult

In [24]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [36]:
# Замена значений целевой переменной
data['class'] = data['class'].replace({'<=50K': 1, '>50K': 0})

# Выделение признаковых и целевых столбцов
X = data.drop(['class'], axis=1)
y = data['class']

print(X.head())
print(y.head())

   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country  
0          2174             0              40  United-States  
1             0             0              13  United-St

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [26]:
# Импорт необходимой функции
from sklearn.metrics import f1_score

# Подготовка предсказаний
predicted = y.mode().values[0]
predicted = [predicted] * len(y)

# Вычисление метрик
accuracy = (y == predicted).mean()
f1_score = f1_score(y, predicted)

print("Accuracy:", accuracy)
print("F1 score:", f1_score)

Accuracy: 0.7607182343065395
F1 score: 0.8640999104619929


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [27]:
from sklearn.impute import SimpleImputer
# Загрузка данных
data = pd.read_csv('adult.csv')

# Определение отсутствующих значений
missing_vals = data.isnull()

# Создание imputer для заполнения отсутствующих значений
imputer = SimpleImputer(strategy='most_frequent')

# Применение imputer к данным
data_filled = imputer.fit_transform(data)

# Преобразование обратно в DataFrame
data_filled = pd.DataFrame(data_filled, columns=data.columns)

print(data_filled.isnull().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [28]:


# Выбор колонок с числовыми данными
numeric_cols = data.select_dtypes(include=[np.number]).columns

# Выбор колонок с категориальными данными
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

Numeric columns: Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')
Categorical columns: Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'class'],
      dtype='object')


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [38]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

data = pd.read_csv('adult.csv')
data['class'] = data['class'].replace({'<=50K': 1, '>50K': 0})

# Выделение признаковых и целевых столбцов
X = data.drop(['class'], axis=1)
y = data['class']

# Определение столбцов для обработки
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
numeric_columns = ['age', 'fnlwgt', 'education-num']

# Пайплайн обработки данных
preprocessor = ColumnTransformer([
    ('cat_encoder', OneHotEncoder(), categorical_columns),
    ('num_scaler', MinMaxScaler(), numeric_columns)
], remainder='passthrough')

# Алгоритмы классификации
models = [
    LogisticRegression(),
    KNeighborsClassifier(),
    LinearSVC()
]

# Кросс-валидация
cv = StratifiedKFold(n_splits=5)
scoring = {'accuracy': 'accuracy', 'f1_macro': 'f1_macro'}
results = {}
for model in models:
    results[model.__class__.__name__] = cross_validate(make_pipeline(preprocessor, model), data, data['class'].values, cv=cv, scoring=scoring)

for model_name, result in results.items():
    accuracy = result['test_accuracy'].mean()
    f1_macro = result['test_f1_macro'].mean()
    print(f"{model_name}:")
    print(f"\tAccuracy: {accuracy:.3f}")
    print(f"\tF1 macro: {f1_macro:.3f}")

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

LogisticRegression:
	Accuracy: nan
	F1 macro: nan
KNeighborsClassifier:
	Accuracy: nan
	F1 macro: nan
LinearSVC:
	Accuracy: nan
	F1 macro: nan


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

data = pd.read_csv('adult.csv')

# Замена значений '?' на самые частые значения
data = data.fillna(data.mode().iloc[0])

# Выделение числовых и категориальных колонок
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

# Пайплайн для числовых колонок
num_pipeline = Pipeline([('imputer', SimpleImputer()),
                         ('scaler', MinMaxScaler())])

# Пайплайн для категориальных колонок
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                         ('encoder', OrdinalEncoder())])

# Комбинация пайплайнов
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_cols),
                                              ('cat', cat_pipeline, categorical_cols)])

# Алгоритмы классификации
models = [LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

# Кросс-валидационные оценки
scores = {}
for model in models:
    pipeline = Pipeline([('preprocessor', preprocessor),
                         ('classifier', model)])
    
    scores[model.__class__.__name__] = cross_val_score(pipeline, data, data['class'],
                                                       scoring='accuracy', cv=5)

for model_name, scores in scores.items():
    print(f"{model_name}:")
    for metric, score in zip(['accuracy'], scores):
        print(f"\t{metric}:\t{score:.3f}")

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

LogisticRegression:
	accuracy:	1.000
KNeighborsClassifier:
	accuracy:	0.966
LinearSVC:
	accuracy:	1.000


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

data = pd.read_csv('adult.csv')

# Выделение числовых и категориальных колонок
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

# Пайплайн для числовых колонок
num_pipeline = Pipeline([('imputer', SimpleImputer()),
                         ('scaler', MinMaxScaler())])

# Пайплайн для категориальных колонок
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                         ('encoder', OrdinalEncoder())])

# Комбинация пайплайнов
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_cols),
                                              ('cat', cat_pipeline, categorical_cols)])

# Алгоритмы классификации
models = [LogisticRegression(), KNeighborsClassifier(), LinearSVC()]

# Кросс-валидационные оценки
scores = {}
for model in models:
    pipeline = Pipeline([('preprocessor', preprocessor),
                         ('classifier', model)])
    
    scores[model.__class__.__name__] = cross_val_score(pipeline, data, data['class'],
                                                       scoring='f1_macro', cv=5)

for model_name, scores in scores.items():
    print(f"{model_name}:")
    for metric, score in zip(['accuracy', 'f1_macro'], scores.mean(axis=0)):
        print(f"\t{metric}:\t{score:.3f}")

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [38]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

data = pd.read_csv('adult.csv')

# Замена значений '?' на самые частые значения
data = data.fillna(data.mode().iloc[0])

# Выделение числовых и категориальных колонок
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

# Пайплайн для числовых колонок
num_pipeline = Pipeline([('imputer', SimpleImputer()),
                         ('scaler', MinMaxScaler())])

# Пайплайн для категориальных колонок
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                         ('encoder', OrdinalEncoder())])

# Комбинация пайплайнов
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_cols),
                                              ('cat', cat_pipeline, categorical_cols)])

# Алгоритмы классификации
models = [RandomForestClassifier(), GradientBoostingClassifier()]

# Кросс-валидационные оценки
scores = {}
for model in models:
    pipeline = Pipeline([('preprocessor', preprocessor),
                         ('classifier', model)])
    
    scores[model.__class__.__name__] = cross_val_score(pipeline, data, data['class'],
                                                       scoring='accuracy', cv=5)

for model_name, scores in scores.items():
    print(f"{model_name}:")
    for metric, score in zip(['accuracy'], scores):
        print(f"\t{metric}:\t{score:.3f}")

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
  File "C:\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "C:\Anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "C:\Anaconda3\lib\site-packages\sklearn\utils\_response.py", line 211, in _get_response_values
    y_pred = prediction_method(X)
  File "C:\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 600, in predict
    Xt = transform.transform(Xt)
  File "C:\Anaconda3\lib\site-packages\sklearn\utils\_set_output.py", line 316, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "C:\Anacon

RandomForestClassifier:
	accuracy:	1.000
GradientBoostingClassifier:
	accuracy:	1.000


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

data = pd.read_csv('adult.csv')

# Выделение числовых и категориальных колонок
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(exclude=[np.number]).columns

# Пайплайн для числовых колонок
num_pipeline = Pipeline([('imputer', SimpleImputer()),
                         ('scaler', MinMaxScaler())])

# Пайплайн для категориальных колонок
cat_pipeline = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                         ('encoder', OrdinalEncoder())])

# Комбинация пайплайнов
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_cols),
                                              ('cat', cat_pipeline, categorical_cols)])
# Создание объектов GridSearchCV
lr_gridsearch = GridSearchCV(Pipeline([('preprocessor', preprocessor),
                                       ('classifier', LogisticRegression())]),
                             param_grid, scoring='accuracy', cv=5, n_jobs=-1)
knn_gridsearch = GridSearchCV(Pipeline([('preprocessor', preprocessor),
                                        ('classifier', KNeighborsClassifier())]),
                              param_grid, scoring='accuracy', cv=5, n_jobs=-1)
svc_gridsearch = GridSearchCV(Pipeline([('preprocessor', preprocessor),
                                        ('classifier', LinearSVC())]),
                              param_grid, scoring='accuracy', cv=5, n_jobs=-1)

# Запуск GridSearchCV для каждого Pipeline
lr_gridsearch.fit(data, data['class'])
knn_gridsearch.fit(data, data['class'])
svc_gridsearch.fit(data, data['class'])

# Получение лучших моделей и их accuracy
lr_best_model = lr_gridsearch.best_estimator_
knn_best_model = knn_gridsearch.best_estimator_
svc_best_model = svc_gridsearch.best_estimator_

lr_accuracy = lr_gridsearch.best_score_
knn_accuracy = knn_gridsearch.best_score_
svc_accuracy = svc_gridsearch.best_score_

# Получение F1-Score для лучших моделей
lr_f1_score = lr_gridsearch.best_score_
knn_f1_score = knn_gridsearch.best_score_
svc_f1_score = svc_gridsearch.best_score_


print("Лучшая модель LogisticRegression:")
print("\t", lr_best_model.get_params())
print("\t Accuracy:", lr_accuracy)
print("\t F1 Score:", lr_f1_score)

print("Лучшая модель KNeighborsClassifier:")
print("\t", knn_best_model.get_params())
print("\t Accuracy:", knn_accuracy)
print("\t F1 Score:", knn_f1_score)

print("Лучшая модель LinearSVC:")
print("\t", svc_best_model.get_params())
print("\t Accuracy:", svc_accuracy)
print("\t F1 Score:", svc_f1_score)

In [19]:
python -m pip install --upgrade scikit-learn

SyntaxError: invalid syntax (Temp/ipykernel_11148/4278391279.py, line 1)